In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json
import os
import re
from typing import Any
from dataclasses import dataclass

In [2]:
def print_grid(grid: list[list[str]]):
    print("\n".join("".join(line) for line in grid))


def plot_grid(
    grid: list[list[str]],
    colors: dict[str, int],
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((len(grid), len(grid[0])))
    for r, row in enumerate(grid):
        for c, char in enumerate(row):
            if char in colors:
                arr[r, c] = colors[char]
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)


def plot_objects(
    object_lists: list[list[tuple[int, int]]],
    colors: list[int],
    x_limit: int,
    y_limit: int,
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((y_limit, x_limit))
    for objects, color in zip(object_lists, colors):
        for obj in objects:
            arr[y_limit - 1 - obj[1], obj[0]] = color
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)

In [3]:
dirs4 = [(-1, 0), (0, 1), (1, 0), (0, -1)]
dirs8 = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

today = os.path.basename(globals()["__vsc_ipynb_file__"]).split(".")[0]  # + "_ex"
today

'day24'

In [4]:
def get_lines() -> list[str]:
    lines = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            lines.append(line.rstrip())
    return lines


def get_grid() -> list[list[str]]:
    grid = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            grid.append([c for c in line.rstrip()])
    return grid


def parse_nums(s: str) -> list[int]:
    return [int(x) for x in re.findall(r"-?\d+", s)]


def get_nums() -> list[list[int]]:
    lines = get_lines()
    return [parse_nums(line) for line in lines]


def is_inside_grid(coords: tuple[int, int], grid: list[list[Any]]) -> bool:
    return coords[0] in range(len(grid)) and coords[1] in range(len(grid[0]))

In [5]:
from typing import Literal
from dataclasses import field


@dataclass
class Wire:
    label: str
    val: bool | None = None
    dependencies: list["Wire"] = field(default_factory=list)
    operator: Literal["AND", "OR", "XOR"] | None = None
    downstream: list["Wire"] = field(default_factory=list)

    def can_activate(self) -> bool:
        if self.val is not None:
            return False
        if len(self.dependencies) != 2:
            return False
        if self.operator == "OR" and (
            self.dependencies[0].val or self.dependencies[1].val
        ):
            self.val = True
            return True
        if self.operator == "AND" and (
            self.dependencies[0].val == False or self.dependencies[1].val == False
        ):
            self.val = False
            return True
        if self.dependencies[0].val is None or self.dependencies[1].val is None:
            return False
        if self.operator == "AND":
            self.val = self.dependencies[0].val and self.dependencies[1].val
            return True
        if self.operator == "OR":
            self.val = self.dependencies[0].val or self.dependencies[1].val
            return True
        self.val = self.dependencies[0].val ^ self.dependencies[1].val
        return True

In [6]:
with open(f"./data/{today}.txt") as f:
    lines = f.read().rstrip()

wires: dict[str, Wire] = {}
activated = deque()
for line in lines.split("\n\n")[0].split("\n"):
    wire = line.split(": ")[0]
    val = line.split(": ")[1].rstrip() == "1"
    wires[wire] = Wire(label=wire, val=val)
    activated.append(wire)

for line in lines.split("\n\n")[1].rstrip().split("\n"):
    first = line[:3]
    second = line.split(" -> ")[0][-3:]
    third = line[-3:]
    operator = line.split(" -> ")[0][4:-4]
    assert operator in ("AND", "OR", "XOR"), operator

    if first not in wires:
        wires[first] = Wire(label=first)
    if second not in wires:
        wires[second] = Wire(label=second)
    if third not in wires:
        wires[third] = Wire(label=third)
    
    wires[first].downstream.append(wires[third])
    wires[second].downstream.append(wires[third])

    wires[third].dependencies = [wires[first], wires[second]]
    wires[third].operator = operator

In [7]:
while len(activated) > 0:
    wire = activated.pop()
    for next_wire in wires[wire].downstream:
        if next_wire.can_activate():
            activated.appendleft(next_wire.label)

In [8]:
z_wires = [(l, w) for (l, w) in wires.items() if l[0] == "z"]
z_wires = reversed(sorted(z_wires, key=lambda x: x[0]))
z_wire_values = [("1" if w.val else "0") for (_, w) in z_wires]
int("".join(z_wire_values), 2)

45923082839246